# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
You can interact with Gemini using a single-turn prompt and response or chat with it in a multi-turn, continuous conversation, even for code understanding and generation.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.7 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    # from google.colab import drive
    # drive.mount('/content/drive')

## Set the environment on GCP Project


In [4]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "You are an AI Specialist",
 "Answer to the question in detail"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [18]:
from vertexai.generative_models import ChatSession

def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    - chat : ChatSession Chat session object
    - prompt : str - The prompt to send to the model.
    - returns: str - The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    print(f"---------------- Current Chat ---------------- ")
    display(Markdown(f"Question : {question}"))
    display(Markdown(f"Answer : {responses.text}"))

    return responses.text

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  print(f"---------------- Chat history ---------------- ")
  print(history)



## Run example

In [19]:

chat = model.start_chat()

get_chat_history(chat)

interactive_chat(chat, "Hello.")


---------------- Chat history ---------------- 

---------------- Current Chat ---------------- 


Question : Hello.

Answer : Hello! 👋

It's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. 


"Hello! 👋\n\nIt's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. \n"

In [20]:
get_chat_history(chat)

interactive_chat(chat, "What are all the colors in a rainbow?.")

---------------- Chat history ---------------- 
Hello. Hello! 👋

It's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. 

---------------- Current Chat ---------------- 


Question : What are all the colors in a rainbow?.

Answer : The colors of a rainbow are often remembered with the acronym **ROYGBIV**:

* **R**ed
* **O**range
* **Y**ellow
* **G**reen
* **B**lue
* **I**ndigo
* **V**iolet 

Some people also include **magenta** as a color in the rainbow, as it's often visible between red and violet. 

It's important to remember that the rainbow is actually a continuous spectrum of colors, and the seven we typically list are just the most prominent.  🌈


"The colors of a rainbow are often remembered with the acronym **ROYGBIV**:\n\n* **R**ed\n* **O**range\n* **Y**ellow\n* **G**reen\n* **B**lue\n* **I**ndigo\n* **V**iolet \n\nSome people also include **magenta** as a color in the rainbow, as it's often visible between red and violet. \n\nIt's important to remember that the rainbow is actually a continuous spectrum of colors, and the seven we typically list are just the most prominent.  🌈\n"

In [22]:
get_chat_history(chat)

interactive_chat(chat, "Why does it appear when it rains?.")


---------------- Chat history ---------------- 
Hello. Hello! 👋

It's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. 
 What are all the colors in a rainbow?. The colors of a rainbow are often remembered with the acronym **ROYGBIV**:

* **R**ed
* **O**range
* **Y**ellow
* **G**reen
* **B**lue
* **I**ndigo
* **V**iolet 

Some people also include **magenta** as a color in the rainbow, as it's often visible between red and violet. 

It's important to remember that the rainbow is actually a continuous spectrum of colors, and the seven we typically list are just the most prominent.  🌈
 Why does it appear when it rains?. That's a great question!  A rainbow appears when sunlight interacts with water droplets in the air, like after a rain shower. Here's a breakdown of what happens:

1. **Sunlight:**  Sunlight is actually made up of all the colors of

Question : Why does it appear when it rains?.

Answer : You're right!  I apologize for repeating myself. I must be getting a little foggy after all those rainbows. 

You're absolutely correct that rainbows appear after rain.  I should have been more specific in my previous explanation:

Rainbows form when sunlight interacts with **water droplets suspended in the air**, which are abundant after a rain shower.   The sunlight is refracted and reflected by these droplets, creating the separation of colors that we see as a rainbow. 

Thanks for pointing out my mistake!  I'm still learning to be a better AI Specialist, and I appreciate your help in keeping me on track. 😊


"You're right!  I apologize for repeating myself. I must be getting a little foggy after all those rainbows. \n\nYou're absolutely correct that rainbows appear after rain.  I should have been more specific in my previous explanation:\n\nRainbows form when sunlight interacts with **water droplets suspended in the air**, which are abundant after a rain shower.   The sunlight is refracted and reflected by these droplets, creating the separation of colors that we see as a rainbow. \n\nThanks for pointing out my mistake!  I'm still learning to be a better AI Specialist, and I appreciate your help in keeping me on track. 😊\n"

In [23]:
# Clear chat history

chat.history.clear()
get_chat_history(chat)

---------------- Chat history ---------------- 



In [24]:
get_chat_history(chat)

interactive_chat(chat, "Why does it appear when it rains?.")

---------------- Chat history ---------------- 

---------------- Current Chat ---------------- 


Question : Why does it appear when it rains?.

Answer : Please clarify your question. You're asking "Why does it appear when it rains?". 

To help me understand what you're asking, please provide more context. For example:

* **What is "it" referring to?**  Are you talking about a specific object, phenomenon, or feeling? 
* **What do you mean by "appear"?** Is it becoming visible, changing appearance, or something else?

Once you give me more information, I can provide a detailed and accurate answer. 


'Please clarify your question. You\'re asking "Why does it appear when it rains?". \n\nTo help me understand what you\'re asking, please provide more context. For example:\n\n* **What is "it" referring to?**  Are you talking about a specific object, phenomenon, or feeling? \n* **What do you mean by "appear"?** Is it becoming visible, changing appearance, or something else?\n\nOnce you give me more information, I can provide a detailed and accurate answer. \n'